In [1]:
# 뉴스 기사 크롤링
import requests
import re
import pandas as pd
import csv
import os
import time
import random
import sys
import json
import logging
from datetime import datetime
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import logging
import pandas as pd
import time
import re
from datetime import datetime

# 웹드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 로깅 설정
logging.basicConfig(level=logging.ERROR, filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

Press = []
Reporter = []
Category = []
SubmissionDate = [] 
SubmissionTime = []
UpdataDate = []
UpdataTime = []
titles = []
articles = []
Size = []
urls = []

url = "http://en.people.cn/"
driver.get(url)

# 정규식 패턴
time_pattern = re.compile(r'(\d{2}:\d{2}), (\w+ \d{2}, \d{4})')

for i in range(1, 11): # 페이지 무한대 설정 가능
    page = f'http://en.people.cn/90785/index{i}.html'
    driver.get(page)
    try:
        for j in range(1, 21):
            
            # 기사 페이지
            headline = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'/html/body/div/div[5]/div[1]/ul/li[{j}]/a'))
            )
            # 큰 기사 링크로 이동
            headline_href = headline.get_attribute('href')
            
            # 새 창 열기
            driver.execute_script("window.open(arguments[0]);", headline_href)
            
            # 새 창으로 전환
            driver.switch_to.window(driver.window_handles[1])
            
            # Press
            Press.append('People.cn')
            
            # reporter
            try: 
                reporter = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'editor'))
                )
                # 기자 이름 추출
                reporter_text = reporter.text
                if '(' in reporter_text and ')' in reporter_text:
                    names = reporter_text.split('(')[1].split(')')[0]
                    names = names.replace('Web editor:', '').strip()
                    reporter_names = [name.strip() for name in names.split(',')]
                    reporter_result = ', '.join(reporter_names)
                else:
                    reporter_result = ''
                Reporter.append(reporter_result)
                
            except Exception as e:
                logging.error(e)
                print(e)
                Reporter.append(None)
                
            # Category
            Category.append('Politics')
            
            # Submission Date
            submission_time_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="origin cf"]/span'))
            )
            submission_time_text = submission_time_element.text
            
            match = time_pattern.search(submission_time_text)
            
            if match:
                time_str = match.group(1)
                date_str = match.group(2)
                date = datetime.strptime(date_str, '%B %d, %Y').strftime('%Y%m%d')
                SubmissionDate.append(date)
                SubmissionTime.append(time_str)
            else:
                SubmissionDate.append(None)
                SubmissionTime.append(None)
                
            # Last Edited Date
            UpdataDate.append(None)
            
            # Last Edited Time
            UpdataTime.append(None)
            
            # titles
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/h1'))
            )
            titles.append(title.text)
            
            # articles
            article_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="w860 d2txtCon cf"]'))
            )
            paragraphs = article_box.find_elements(By.TAG_NAME, 'p')
            article = ''
            for paragraph in paragraphs:
                article += paragraph.text + '\n'
            articles.append(article)
            
            # Size
            Size.append(len(article))
            
            # url
            urls.append(headline_href)
            
            print(f'{j}번째 기사 크롤링 완료')
            
            # 새 창 닫기
            driver.close()
            
            # 원래 창으로 돌아가기
            driver.switch_to.window(driver.window_handles[0])
            
    except Exception as e:
        logging.error(e)
        print(e)
        # 다음 단계로 넘어감 
        continue
    

# Press, Reporter, Category, SubmissionDate, SubmissionTime, UpdataDate, UpdataTime, titles, articles, Size, urls의 길이를 확인
array_lengths = [len(Press), len(Reporter), len(Category), len(SubmissionDate), len(SubmissionTime), len(UpdataDate), len(UpdataTime), len(titles), len(articles), len(Size), len(urls)]
max_length = max(array_lengths)

# 각 배열의 길이를 최대 길이에 맞추고, 빈 값을 NULL로 채우기
Press += [None] * (max_length - len(Press))
Reporter += [None] * (max_length - len(Reporter))
Category += [None] * (max_length - len(Category))
SubmissionDate += [None] * (max_length - len(SubmissionDate))
SubmissionTime += [None] * (max_length - len(SubmissionTime))
UpdataDate += [None] * (max_length - len(UpdataDate))
UpdataTime += [None] * (max_length - len(UpdataTime))
titles += [None] * (max_length - len(titles))
articles += [None] * (max_length - len(articles))
Size += [None] * (max_length - len(Size))
urls += [None] * (max_length - len(urls))


# 데이터 프레임 생성
politics = pd.DataFrame({
    'Press': Press,
    'Reporter': Reporter,
    'Category': Category,
    'Submission Date': SubmissionDate,
    'Submission Time': SubmissionTime,
    'Last Edited Date': UpdataDate,
    'Last Edited Time': UpdataTime,
    'Article Title': titles,
    'Article Body': articles,
    'Size': Size,
    'url': urls
})    
    

# csv 파일로 저장
politics.to_csv('China_Politics.csv', index=False)

print('크롤링 완료')


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import logging
import pandas as pd
import time
import re
from datetime import datetime

# 웹드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 로깅 설정
logging.basicConfig(level=logging.ERROR, filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

Press = []
Reporter = []
Category = []
SubmissionDate = [] 
SubmissionTime = []
UpdataDate = []
UpdataTime = []
titles = []
articles = []
Size = []
urls = []

url = "http://en.people.cn/"
driver.get(url)

# 정규식 패턴
time_pattern = re.compile(r'(\d{2}:\d{2}), (\w+ \d{2}, \d{4})')

for i in range(1, 11): # 페이지 무한대 설정 가능
    page = f'http://en.people.cn/business/index{i}.html'
    driver.get(page)
    try:
        for j in range(1, 21):
            
            # 기사 페이지
            headline = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'/html/body/div/div[5]/div[1]/ul/li[{j}]/a'))
            )
            # 큰 기사 링크로 이동
            headline_href = headline.get_attribute('href')
            
            # 새 창 열기
            driver.execute_script("window.open(arguments[0]);", headline_href)
            
            # 새 창으로 전환
            driver.switch_to.window(driver.window_handles[1])
            
            # Press
            Press.append('People.cn')
            
            # reporter
            try: 
                reporter = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'editor'))
                )
                # 기자 이름 추출
                reporter_text = reporter.text
                if '(' in reporter_text and ')' in reporter_text:
                    names = reporter_text.split('(')[1].split(')')[0]
                    names = names.replace('Web editor:', '').strip()
                    reporter_names = [name.strip() for name in names.split(',')]
                    reporter_result = ', '.join(reporter_names)
                else:
                    reporter_result = ''
                Reporter.append(reporter_result)
                
            except Exception as e:
                logging.error(e)
                print(e)
                Reporter.append(None)
                
            # Category
            Category.append('Economy')
            
            # Submission Date
            submission_time_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="origin cf"]/span'))
            )
            submission_time_text = submission_time_element.text
            
            match = time_pattern.search(submission_time_text)
            
            if match:
                time_str = match.group(1)
                date_str = match.group(2)
                date = datetime.strptime(date_str, '%B %d, %Y').strftime('%Y%m%d')
                
                SubmissionDate.append(int(date))
                SubmissionTime.append(int(time_str.replace(':', '')))
            else:
                SubmissionDate.append(None)
                SubmissionTime.append(None)
                
            # Last Edited Date
            UpdataDate.append(None)
            
            # Last Edited Time
            UpdataTime.append(None)
            
            # titles
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/h1'))
            )
            titles.append(title.text)
            
            # articles
            article_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="w860 d2txtCon cf"]'))
            )
            paragraphs = article_box.find_elements(By.TAG_NAME, 'p')
            article = ''
            for paragraph in paragraphs:
                article += paragraph.text + '\n'
            articles.append(article)
            
            # Size
            Size.append(len(article))
            
            # url
            urls.append(headline_href)
            
            print(f'{j}번째 기사 크롤링 완료')
            
            # 새 창 닫기
            driver.close()
            
            # 원래 창으로 돌아가기
            driver.switch_to.window(driver.window_handles[0])
            
    except Exception as e:
        logging.error(e)
        print(e)
        # 다음 단계로 넘어감 
        continue
    

# Press, Reporter, Category, SubmissionDate, SubmissionTime, UpdataDate, UpdataTime, titles, articles, Size, urls의 길이를 확인
array_lengths = [len(Press), len(Reporter), len(Category), len(SubmissionDate), len(SubmissionTime), len(UpdataDate), len(UpdataTime), len(titles), len(articles), len(Size), len(urls)]
max_length = max(array_lengths)

# 각 배열의 길이를 최대 길이에 맞추고, 빈 값을 NULL로 채우기
Press += [None] * (max_length - len(Press))
Reporter += [None] * (max_length - len(Reporter))
Category += [None] * (max_length - len(Category))
SubmissionDate += [None] * (max_length - len(SubmissionDate))
SubmissionTime += [None] * (max_length - len(SubmissionTime))
UpdataDate += [None] * (max_length - len(UpdataDate))
UpdataTime += [None] * (max_length - len(UpdataTime))
titles += [None] * (max_length - len(titles))
articles += [None] * (max_length - len(articles))
Size += [None] * (max_length - len(Size))
urls += [None] * (max_length - len(urls))


# 데이터 프레임 생성
politics = pd.DataFrame({
    'Press': Press,
    'Reporter': Reporter,
    'Category': Category,
    'Submission Date': SubmissionDate,
    'Submission Time': SubmissionTime,
    'Last Edited Date': UpdataDate,
    'Last Edited Time': UpdataTime,
    'Article Title': titles,
    'Article Body': articles,
    'Size': Size,
    'url': urls
})    
    

# csv 파일로 저장
politics.to_csv('China_Economy.csv', index=False)

print('크롤링 완료')


1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
1

In [42]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import logging
import pandas as pd
import time
import re
from datetime import datetime

# 웹드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 로깅 설정
logging.basicConfig(level=logging.ERROR, filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

Press = []
Reporter = []
Category = []
SubmissionDate = [] 
SubmissionTime = []
UpdataDate = []
UpdataTime = []
titles = []
articles = []
Size = []
urls = []

url = "http://en.people.cn/"
driver.get(url)

# 정규식 패턴
time_pattern = re.compile(r'(\d{2}:\d{2}), (\w+ \d{2}, \d{4})')

for i in range(1, 15): # 페이지 무한대 설정 가능
    page = f'http://en.people.cn/90882/index{i}.html'
    driver.get(page)
    try:
        for j in range(1, 21):
            
            # 기사 페이지
            headline = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'/html/body/div/div[5]/div[1]/ul/li[{j}]/a'))
            )
            # 큰 기사 링크로 이동
            headline_href = headline.get_attribute('href')
            
            # 새 창 열기
            driver.execute_script("window.open(arguments[0]);", headline_href)
            
            # 새 창으로 전환
            driver.switch_to.window(driver.window_handles[1])
            
            # Press
            Press.append('People.cn')
            
            # reporter
            try: 
                reporter = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'editor'))
                )
                # 기자 이름 추출
                reporter_text = reporter.text
                if '(' in reporter_text and ')' in reporter_text:
                    names = reporter_text.split('(')[1].split(')')[0]
                    names = names.replace('Web editor:', '').strip()
                    reporter_names = [name.strip() for name in names.split(',')]
                    reporter_result = ', '.join(reporter_names)
                else:
                    reporter_result = ''
                Reporter.append(reporter_result)
                
            except Exception as e:
                logging.error(e)
                print(e)
                Reporter.append(None)
                
            # Category  
            Category.append('Society')
            
            # Submission Date
            submission_time_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="origin cf"]/span'))
            )
            submission_time_text = submission_time_element.text
            
            match = time_pattern.search(submission_time_text)
            
            if match:
                time_str = match.group(1)
                date_str = match.group(2)
                date = datetime.strptime(date_str, '%B %d, %Y').strftime('%Y%m%d')
                
                SubmissionDate.append(int(date))
                SubmissionTime.append(int(time_str.replace(':', '')))
            else:
                SubmissionDate.append(None)
                SubmissionTime.append(None)
                
            # Last Edited Date
            UpdataDate.append(None)
            
            # Last Edited Time
            UpdataTime.append(None)
            
            # titles
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/h1'))
            )
            titles.append(title.text)
            
            # articles
            article_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="w860 d2txtCon cf"]'))
            )
            paragraphs = article_box.find_elements(By.TAG_NAME, 'p')
            article = ''
            for paragraph in paragraphs:
                article += paragraph.text + '\n'
            articles.append(article)
            
            # Size
            Size.append(len(article))
            
            # url
            urls.append(headline_href)
            
            print(f'{j}번째 기사 크롤링 완료')
            
            # 새 창 닫기
            driver.close()
            
            # 원래 창으로 돌아가기
            driver.switch_to.window(driver.window_handles[0])
            
    except Exception as e:
        logging.error(e)
        print(e)
        # 다음 단계로 넘어감 
        continue
    print(f'{i}번째 페이지 크롤링 완료')
    
    

# Press, Reporter, Category, SubmissionDate, SubmissionTime, UpdataDate, UpdataTime, titles, articles, Size, urls의 길이를 확인
array_lengths = [len(Press), len(Reporter), len(Category), len(SubmissionDate), len(SubmissionTime), len(UpdataDate), len(UpdataTime), len(titles), len(articles), len(Size), len(urls)]
max_length = max(array_lengths)

# 각 배열의 길이를 최대 길이에 맞추고, 빈 값을 NULL로 채우기
Press += [None] * (max_length - len(Press))
Reporter += [None] * (max_length - len(Reporter))
Category += [None] * (max_length - len(Category))
SubmissionDate += [None] * (max_length - len(SubmissionDate))
SubmissionTime += [None] * (max_length - len(SubmissionTime))
UpdataDate += [None] * (max_length - len(UpdataDate))
UpdataTime += [None] * (max_length - len(UpdataTime))
titles += [None] * (max_length - len(titles))
articles += [None] * (max_length - len(articles))
Size += [None] * (max_length - len(Size))
urls += [None] * (max_length - len(urls))


# 데이터 프레임 생성
politics = pd.DataFrame({
    'Press': Press,
    'Reporter': Reporter,
    'Category': Category,
    'Submission Date': SubmissionDate,
    'Submission Time': SubmissionTime,
    'Last Edited Date': UpdataDate,
    'Last Edited Time': UpdataTime,
    'Article Title': titles,
    'Article Body': articles,
    'Size': Size,
    'url': urls
})    
    

# csv 파일로 저장
politics.to_csv('China_Society.csv', index=False)

print('크롤링 완료')


1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
Message: 
Stacktrace:
	GetHandleVerifier [0x004CB8E3+45827]
	(No symbol) [0x0045DCC4]
	(No symbol) [0x0035150F]
	(No symbol) [0x003920BC]
	(No symbol) [0x0039216B]
	(No symbol) [0x003CE0F2]
	(No symbol) [0x003B2E44]
	(No symbol) [0x003CC034]
	(No symbol) [0x003B

In [43]:
so = pd.read_csv('China_Society.csv')
so

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Zhang Kaiwei, Liang Jun",Society,20240613.0,1012.0,NaN,NaN,Alerts maintain as heat wave lingers in multip...,"BEIJING, June 12 (Xinhua) -- A lingering heat ...",1878.0,http://en.people.cn/n3/2024/0613/c90000-201808...
1,People.cn,"Hongyu, Liang Jun",Society,20240612.0,1549.0,NaN,NaN,Giant panda Fu Bao meets the public in China,\nThe beloved giant panda Fu Bao officially me...,2596.0,http://en.people.cn/n3/2024/0612/c98649-201805...
2,People.cn,"Hongyu, Liang Jun",Society,20240612.0,1432.0,NaN,NaN,Village in SE China turns green mountains into...,\nPhoto shows a sample of the forest carbon cr...,2526.0,http://en.people.cn/n3/2024/0612/c98649-201805...
3,People.cn,"Hongyu, Liang Jun",Society,20240612.0,1033.0,NaN,NaN,"China builds ""photovoltaic Great Wall"" against...","Construction of a ""photovoltaic Great Wall,” c...",1221.0,http://en.people.cn/n3/2024/0612/c312369-20180...
4,People.cn,"Tian Yi, Liang Jun",Society,20240612.0,949.0,NaN,NaN,Mountain county changes its spots to ensure le...,"\nHuang Qiaowen, executive director of the Chi...",10131.0,http://en.people.cn/n3/2024/0612/c90000-201803...
5,People.cn,"Hongyu, Liang Jun",Society,20240612.0,928.0,NaN,NaN,NE China's Jilin promotes mixed rice-crayfish ...,\nA farmer operates an agricultural machine to...,3541.0,http://en.people.cn/n3/2024/0612/c90000-201803...
6,People.cn,"Zhang Kaiwei, Liang Jun",Society,20240612.0,924.0,NaN,NaN,China's 2024 legal professional exam to open f...,"BEIJING, June 11 (Xinhua) -- China's national ...",716.0,http://en.people.cn/n3/2024/0612/c90000-201803...
7,People.cn,"Zhang Kaiwei, Liang Jun",Society,20240612.0,922.0,NaN,NaN,Xizang Story: World's highest peak elevates li...,"LHASA, June 11 (Xinhua) -- It was already 10 a...",3619.0,http://en.people.cn/n3/2024/0612/c90000-201803...
8,People.cn,"Zhang Kaiwei, Liang Jun",Society,20240612.0,919.0,NaN,NaN,China launches recruitment campaign for gradua...,\nA recruiting representative (R) talks with j...,1244.0,http://en.people.cn/n3/2024/0612/c90000-201803...
9,People.cn,"Zhang Kaiwei, Liang Jun",Society,20240612.0,917.0,NaN,NaN,Victims in NE China knife attack sustain no li...,"CHANGCHUN, June 11 (Xinhua) -- Police in north...",874.0,http://en.people.cn/n3/2024/0612/c90000-201803...


In [44]:
# Submission Date	Submission Time 둘 다 정수로 변환 , 0채우기

so['Submission Date'] = so['Submission Date'].fillna(0).astype(int)

so['Submission Time'] = so['Submission Time'].fillna(0).astype(int)

so

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Zhang Kaiwei, Liang Jun",Society,20240613,1012,NaN,NaN,Alerts maintain as heat wave lingers in multip...,"BEIJING, June 12 (Xinhua) -- A lingering heat ...",1878.0,http://en.people.cn/n3/2024/0613/c90000-201808...
1,People.cn,"Hongyu, Liang Jun",Society,20240612,1549,NaN,NaN,Giant panda Fu Bao meets the public in China,\nThe beloved giant panda Fu Bao officially me...,2596.0,http://en.people.cn/n3/2024/0612/c98649-201805...
2,People.cn,"Hongyu, Liang Jun",Society,20240612,1432,NaN,NaN,Village in SE China turns green mountains into...,\nPhoto shows a sample of the forest carbon cr...,2526.0,http://en.people.cn/n3/2024/0612/c98649-201805...
3,People.cn,"Hongyu, Liang Jun",Society,20240612,1033,NaN,NaN,"China builds ""photovoltaic Great Wall"" against...","Construction of a ""photovoltaic Great Wall,” c...",1221.0,http://en.people.cn/n3/2024/0612/c312369-20180...
4,People.cn,"Tian Yi, Liang Jun",Society,20240612,949,NaN,NaN,Mountain county changes its spots to ensure le...,"\nHuang Qiaowen, executive director of the Chi...",10131.0,http://en.people.cn/n3/2024/0612/c90000-201803...
5,People.cn,"Hongyu, Liang Jun",Society,20240612,928,NaN,NaN,NE China's Jilin promotes mixed rice-crayfish ...,\nA farmer operates an agricultural machine to...,3541.0,http://en.people.cn/n3/2024/0612/c90000-201803...
6,People.cn,"Zhang Kaiwei, Liang Jun",Society,20240612,924,NaN,NaN,China's 2024 legal professional exam to open f...,"BEIJING, June 11 (Xinhua) -- China's national ...",716.0,http://en.people.cn/n3/2024/0612/c90000-201803...
7,People.cn,"Zhang Kaiwei, Liang Jun",Society,20240612,922,NaN,NaN,Xizang Story: World's highest peak elevates li...,"LHASA, June 11 (Xinhua) -- It was already 10 a...",3619.0,http://en.people.cn/n3/2024/0612/c90000-201803...
8,People.cn,"Zhang Kaiwei, Liang Jun",Society,20240612,919,NaN,NaN,China launches recruitment campaign for gradua...,\nA recruiting representative (R) talks with j...,1244.0,http://en.people.cn/n3/2024/0612/c90000-201803...
9,People.cn,"Zhang Kaiwei, Liang Jun",Society,20240612,917,NaN,NaN,Victims in NE China knife attack sustain no li...,"CHANGCHUN, June 11 (Xinhua) -- Police in north...",874.0,http://en.people.cn/n3/2024/0612/c90000-201803...


In [48]:

so.to_csv('China_Society.csv', index=False)



In [50]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import logging
import pandas as pd
import time
import re
from datetime import datetime

# 웹드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 로깅 설정
logging.basicConfig(level=logging.ERROR, filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

Press = []
Reporter = []
Category = []
SubmissionDate = [] 
SubmissionTime = []
UpdataDate = []
UpdataTime = []
titles = []
articles = []
Size = []
urls = []

url = "http://en.people.cn/"
driver.get(url)

# 정규식 패턴
time_pattern = re.compile(r'(\d{2}:\d{2}), (\w+ \d{2}, \d{4})')

for i in range(1, 11): # 페이지 무한대 설정 가능
    page = f'http://en.people.cn/90777/index{i}.html'
    driver.get(page)
    try:
        for j in range(1, 21):
            
            # 기사 페이지
            headline = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'/html/body/div/div[5]/div[1]/ul/li[{j}]/a'))
            )
            # 큰 기사 링크로 이동
            headline_href = headline.get_attribute('href')
            
            # 새 창 열기
            driver.execute_script("window.open(arguments[0]);", headline_href)
            
            # 새 창으로 전환
            driver.switch_to.window(driver.window_handles[1])
            
            # Press
            Press.append('People.cn')
            
            # reporter
            try: 
                reporter = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'editor'))
                )
                # 기자 이름 추출
                reporter_text = reporter.text
                if '(' in reporter_text and ')' in reporter_text:
                    names = reporter_text.split('(')[1].split(')')[0]
                    names = names.replace('Web editor:', '').strip()
                    reporter_names = [name.strip() for name in names.split(',')]
                    reporter_result = ', '.join(reporter_names)
                else:
                    reporter_result = ''
                Reporter.append(reporter_result)
                
            except Exception as e:
                logging.error(e)
                print(e)
                Reporter.append(None)

            # Category  
            Category.append('World')
            
            # Submission Date
            submission_time_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="origin cf"]/span'))
            )
            submission_time_text = submission_time_element.text
            
            match = time_pattern.search(submission_time_text)
            
            if match:
                time_str = match.group(1)
                date_str = match.group(2)
                date = datetime.strptime(date_str, '%B %d, %Y').strftime('%Y%m%d')
                
                SubmissionDate.append(int(date))
                SubmissionTime.append(int(time_str.replace(':', '')))
            else:
                SubmissionDate.append(None)
                SubmissionTime.append(None)
                
            # Last Edited Date
            UpdataDate.append(None)
            
            # Last Edited Time
            UpdataTime.append(None)
            
            # titles
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/h1'))
            )
            titles.append(title.text)
            
            # articles
            article_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="w860 d2txtCon cf"]'))
            )
            paragraphs = article_box.find_elements(By.TAG_NAME, 'p')
            article = ''
            for paragraph in paragraphs:
                article += paragraph.text + '\n'
            articles.append(article)
            
            # Size
            Size.append(len(article))
            
            # url
            urls.append(headline_href)
            
            print(f'{j}번째 기사 크롤링 완료')
            
            # 새 창 닫기
            driver.close()
            
            # 원래 창으로 돌아가기
            driver.switch_to.window(driver.window_handles[0])
            
    except Exception as e:
        logging.error(e)
        print(e)
        # 다음 단계로 넘어감 
        continue
    print(f'{i}번째 페이지 크롤링 완료')
    
    

# Press, Reporter, Category, SubmissionDate, SubmissionTime, UpdataDate, UpdataTime, titles, articles, Size, urls의 길이를 확인
array_lengths = [len(Press), len(Reporter), len(Category), len(SubmissionDate), len(SubmissionTime), len(UpdataDate), len(UpdataTime), len(titles), len(articles), len(Size), len(urls)]
max_length = max(array_lengths)

# 각 배열의 길이를 최대 길이에 맞추고, 빈 값을 NULL로 채우기
Press += [None] * (max_length - len(Press))
Reporter += [None] * (max_length - len(Reporter))
Category += [None] * (max_length - len(Category))
SubmissionDate += [None] * (max_length - len(SubmissionDate))
SubmissionTime += [None] * (max_length - len(SubmissionTime))
UpdataDate += [None] * (max_length - len(UpdataDate))
UpdataTime += [None] * (max_length - len(UpdataTime))
titles += [None] * (max_length - len(titles))
articles += [None] * (max_length - len(articles))
Size += [None] * (max_length - len(Size))
urls += [None] * (max_length - len(urls))


# 데이터 프레임 생성
politics = pd.DataFrame({
    'Press': Press,
    'Reporter': Reporter,
    'Category': Category,
    'Submission Date': SubmissionDate,
    'Submission Time': SubmissionTime,
    'Last Edited Date': UpdataDate,
    'Last Edited Time': UpdataTime,
    'Article Title': titles,
    'Article Body': articles,
    'Size': Size,
    'url': urls
})    
    

# csv 파일로 저장
politics.to_csv('China_World.csv', index=False)

print('크롤링 완료')


1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
1번째 페이지 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=125.0.6422.142)
Stacktrace:
	GetHandleVerifier [0x004CB8E3+45827]
	(No symbol) [0x0045DCC4]
	(No symbol) [0x0035150F]
	(No symbol) [0x0033D496]
	(No symbol) [0x0033D311]
	(No symbol) [0x0033B739]
	(No symbol) [0x0033BD1D]
	(No symbol) [0x0034802A]
	(No symbol) [0x0035BF01]
	(No symbol) [0x003603A6]
	(No symbol) [0x0033C38C]
	(No symbol) [0x0035BCE3]
	(No symbol) [0x003CBF53]
	(No symbol) [0x003B2B96]
	(No symbol

In [51]:
wo = pd.read_csv('China_World.csv')
wo

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Zhang Kaiwei, Liang Jun",World,20240613.0,931.0,NaN,NaN,UC Los Angeles appoints new chancellor in afte...,\nAcademic workers protest at the University o...,3340.0,http://en.people.cn/n3/2024/0613/c90000-201807...
1,People.cn,"Zhang Kaiwei, Liang Jun",World,20240613.0,929.0,NaN,NaN,Crucial crossing on Libya-Tunisia border to pa...,\nLibyan Interior Minister Emad al-Tarabelsi (...,1555.0,http://en.people.cn/n3/2024/0613/c90000-201807...
2,People.cn,"Zhang Kaiwei, Zhong Wenxing",World,20240612.0,1639.0,NaN,NaN,160 rockets fired at northern Israel -- Israel...,"JERUSALEM, June 12 (Xinhua) -- About 160 rocke...",225.0,http://en.people.cn/n3/2024/0612/c90000-201806...
3,People.cn,"Zhang Kaiwei, Liang Jun",World,20240612.0,859.0,NaN,NaN,Hunter Biden found guilty on three felony gun ...,\nHunter Biden is pictured at the White House ...,4084.0,http://en.people.cn/n3/2024/0612/c90000-201802...
4,People.cn,"Zhang Kaiwei, Liang Jun",World,20240612.0,855.0,NaN,NaN,About 25 pro-Palestinian protesters arrested a...,\nPolice officers detain a pro-Palestinian pro...,7722.0,http://en.people.cn/n3/2024/0612/c90000-201802...
...,...,...,...,...,...,...,...,...,...,...,...
151,People.cn,"Zhang Kaiwei, Liang Jun",World,20240520.0,905.0,NaN,NaN,Accusations against UNRWA proven false: Jordan FM,\nUN Relief and Works Agency for Palestine Ref...,2531.0,http://en.people.cn/n3/2024/0520/c90000-201715...
152,People.cn,"Zhang Kaiwei, Liang Jun",World,20240520.0,807.0,NaN,NaN,"Fog, rain hamper search for Iran's president a...","TEHRAN, May 19 (Xinhua) -- A hard landing happ...",3913.0,http://en.people.cn/n3/2024/0520/c90000-201714...
153,People.cn,NaN,World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,People.cn,NaN,World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:

wo['Submission Date'] = wo['Submission Date'].fillna(0).astype(int)

wo['Submission Time'] = wo['Submission Time'].fillna(0).astype(int)

wo

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Zhang Kaiwei, Liang Jun",World,20240613,931,NaN,NaN,UC Los Angeles appoints new chancellor in afte...,\nAcademic workers protest at the University o...,3340.0,http://en.people.cn/n3/2024/0613/c90000-201807...
1,People.cn,"Zhang Kaiwei, Liang Jun",World,20240613,929,NaN,NaN,Crucial crossing on Libya-Tunisia border to pa...,\nLibyan Interior Minister Emad al-Tarabelsi (...,1555.0,http://en.people.cn/n3/2024/0613/c90000-201807...
2,People.cn,"Zhang Kaiwei, Zhong Wenxing",World,20240612,1639,NaN,NaN,160 rockets fired at northern Israel -- Israel...,"JERUSALEM, June 12 (Xinhua) -- About 160 rocke...",225.0,http://en.people.cn/n3/2024/0612/c90000-201806...
3,People.cn,"Zhang Kaiwei, Liang Jun",World,20240612,859,NaN,NaN,Hunter Biden found guilty on three felony gun ...,\nHunter Biden is pictured at the White House ...,4084.0,http://en.people.cn/n3/2024/0612/c90000-201802...
4,People.cn,"Zhang Kaiwei, Liang Jun",World,20240612,855,NaN,NaN,About 25 pro-Palestinian protesters arrested a...,\nPolice officers detain a pro-Palestinian pro...,7722.0,http://en.people.cn/n3/2024/0612/c90000-201802...
...,...,...,...,...,...,...,...,...,...,...,...
151,People.cn,"Zhang Kaiwei, Liang Jun",World,20240520,905,NaN,NaN,Accusations against UNRWA proven false: Jordan FM,\nUN Relief and Works Agency for Palestine Ref...,2531.0,http://en.people.cn/n3/2024/0520/c90000-201715...
152,People.cn,"Zhang Kaiwei, Liang Jun",World,20240520,807,NaN,NaN,"Fog, rain hamper search for Iran's president a...","TEHRAN, May 19 (Xinhua) -- A hard landing happ...",3913.0,http://en.people.cn/n3/2024/0520/c90000-201714...
153,People.cn,NaN,World,0,0,NaN,NaN,NaN,NaN,NaN,NaN
154,People.cn,NaN,World,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# 153~155 행 제거

wo = wo.drop([153, 154, 155])

wo

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Zhang Kaiwei, Liang Jun",World,20240613,931,NaN,NaN,UC Los Angeles appoints new chancellor in afte...,\nAcademic workers protest at the University o...,3340.0,http://en.people.cn/n3/2024/0613/c90000-201807...
1,People.cn,"Zhang Kaiwei, Liang Jun",World,20240613,929,NaN,NaN,Crucial crossing on Libya-Tunisia border to pa...,\nLibyan Interior Minister Emad al-Tarabelsi (...,1555.0,http://en.people.cn/n3/2024/0613/c90000-201807...
2,People.cn,"Zhang Kaiwei, Zhong Wenxing",World,20240612,1639,NaN,NaN,160 rockets fired at northern Israel -- Israel...,"JERUSALEM, June 12 (Xinhua) -- About 160 rocke...",225.0,http://en.people.cn/n3/2024/0612/c90000-201806...
3,People.cn,"Zhang Kaiwei, Liang Jun",World,20240612,859,NaN,NaN,Hunter Biden found guilty on three felony gun ...,\nHunter Biden is pictured at the White House ...,4084.0,http://en.people.cn/n3/2024/0612/c90000-201802...
4,People.cn,"Zhang Kaiwei, Liang Jun",World,20240612,855,NaN,NaN,About 25 pro-Palestinian protesters arrested a...,\nPolice officers detain a pro-Palestinian pro...,7722.0,http://en.people.cn/n3/2024/0612/c90000-201802...
...,...,...,...,...,...,...,...,...,...,...,...
148,People.cn,"Zhang Kaiwei, Liang Jun",World,20240520,1107,NaN,NaN,Ethiopian Airlines inaugurates Chinese-built t...,\nTourists are pictured at the newly-built dom...,4340.0,http://en.people.cn/n3/2024/0520/c90000-201717...
149,People.cn,"Zhang Kaiwei, Liang Jun",World,20240520,1009,NaN,NaN,"Feature: 21 years after U.S.-led invasion, Ira...","\nPeople visit the Iraq Museum in Baghdad, Ira...",5375.0,http://en.people.cn/n3/2024/0520/c90000-201716...
150,People.cn,"Zhang Kaiwei, Liang Jun",World,20240520,908,NaN,NaN,Death toll rises to 31 in Israeli raids on cen...,\nPalestinians inspect the rubble after an Isr...,2133.0,http://en.people.cn/n3/2024/0520/c90000-201715...
151,People.cn,"Zhang Kaiwei, Liang Jun",World,20240520,905,NaN,NaN,Accusations against UNRWA proven false: Jordan FM,\nUN Relief and Works Agency for Palestine Ref...,2531.0,http://en.people.cn/n3/2024/0520/c90000-201715...


In [54]:
wo.to_csv('China_World.csv', index=False)



In [59]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import logging
import pandas as pd
import time
import re
from datetime import datetime

# 웹드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 로깅 설정
logging.basicConfig(level=logging.ERROR, filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

Press = []
Reporter = []
Category = []
SubmissionDate = [] 
SubmissionTime = []
UpdataDate = []
UpdataTime = []
titles = []
articles = []
Size = []
urls = []

url = "http://en.people.cn/"
driver.get(url)

# 정규식 패턴
time_pattern = re.compile(r'(\d{2}:\d{2}), (\w+ \d{2}, \d{4})')

for i in range(1, 11): # 페이지 무한대 설정 가능
    page = f'http://en.people.cn/90782/index{i}.html'
    driver.get(page)
    try:
        for j in range(1, 21):
            
            # 기사 페이지
            headline = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'/html/body/div/div[5]/div[1]/ul/li[{j}]/a'))
            )
            # 큰 기사 링크로 이동
            headline_href = headline.get_attribute('href')
            
            # 새 창 열기
            driver.execute_script("window.open(arguments[0]);", headline_href)
            
            # 새 창으로 전환
            driver.switch_to.window(driver.window_handles[1])
            
            # Press
            Press.append('People.cn')
            
            # reporter
            try: 
                reporter = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'editor'))
                )
                # 기자 이름 추출
                reporter_text = reporter.text
                if '(' in reporter_text and ')' in reporter_text:
                    names = reporter_text.split('(')[1].split(')')[0]
                    names = names.replace('Web editor:', '').strip()
                    reporter_names = [name.strip() for name in names.split(',')]
                    reporter_result = ', '.join(reporter_names)
                else:
                    reporter_result = ''
                Reporter.append(reporter_result)
                
            except Exception as e:
                logging.error(e)
                print(e)
                Reporter.append(None)

            # Category  
            Category.append('Culture')
            
            # Submission Date
            submission_time_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="origin cf"]/span'))
            )
            submission_time_text = submission_time_element.text
            
            match = time_pattern.search(submission_time_text)
            
            if match:
                time_str = match.group(1)
                date_str = match.group(2)
                date = datetime.strptime(date_str, '%B %d, %Y').strftime('%Y%m%d')
                
                SubmissionDate.append(int(date))
                SubmissionTime.append(int(time_str.replace(':', '')))
            else:
                SubmissionDate.append(None)
                SubmissionTime.append(None)
                
            # Last Edited Date
            UpdataDate.append(None)
            
            # Last Edited Time
            UpdataTime.append(None)
            
            # titles
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/h1'))
            )
            titles.append(title.text)
            
            # articles
            article_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="w860 d2txtCon cf"]'))
            )
            paragraphs = article_box.find_elements(By.TAG_NAME, 'p')
            article = ''
            for paragraph in paragraphs:
                article += paragraph.text + '\n'
            articles.append(article)
            
            # Size
            Size.append(len(article))
            
            # url
            urls.append(headline_href)
            
            print(f'{j}번째 기사 크롤링 완료')
            
            # 새 창 닫기
            driver.close()
            
            # 원래 창으로 돌아가기
            driver.switch_to.window(driver.window_handles[0])
            
    except Exception as e:
        logging.error(e)
        print(e)
        # 다음 단계로 넘어감 
        continue
    print(f'{i}번째 페이지 크롤링 완료')
    
    

# Press, Reporter, Category, SubmissionDate, SubmissionTime, UpdataDate, UpdataTime, titles, articles, Size, urls의 길이를 확인
array_lengths = [len(Press), len(Reporter), len(Category), len(SubmissionDate), len(SubmissionTime), len(UpdataDate), len(UpdataTime), len(titles), len(articles), len(Size), len(urls)]
max_length = max(array_lengths)

# 각 배열의 길이를 최대 길이에 맞추고, 빈 값을 NULL로 채우기
Press += [None] * (max_length - len(Press))
Reporter += [None] * (max_length - len(Reporter))
Category += [None] * (max_length - len(Category))
SubmissionDate += [None] * (max_length - len(SubmissionDate))
SubmissionTime += [None] * (max_length - len(SubmissionTime))
UpdataDate += [None] * (max_length - len(UpdataDate))
UpdataTime += [None] * (max_length - len(UpdataTime))
titles += [None] * (max_length - len(titles))
articles += [None] * (max_length - len(articles))
Size += [None] * (max_length - len(Size))
urls += [None] * (max_length - len(urls))


# 데이터 프레임 생성
politics = pd.DataFrame({
    'Press': Press,
    'Reporter': Reporter,
    'Category': Category,
    'Submission Date': SubmissionDate,
    'Submission Time': SubmissionTime,
    'Last Edited Date': UpdataDate,
    'Last Edited Time': UpdataTime,
    'Article Title': titles,
    'Article Body': articles,
    'Size': Size,
    'url': urls
})    
    

# csv 파일로 저장
politics.to_csv('China_Culture.csv', index=False)

print('크롤링 완료')


1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
1번째 페이지 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
Message: 
Stacktrace:
	GetHandleVerifier [0x004CB8E3+45827]
	(No symbol) [0x0045DCC4]
	(No symbol) [0x0035150F]
	(No symbol) [0x003920BC]
	(No symbol) [0x0039216B]
	(No symbol) [0x003CE0F2]
	(No symbol) [0x003B2E44]
	(No symbol) [0x003CC034]
	(No symbol) [0x003B2B96]
	(No symbol) [0x00386998]
	(No symbol) [0x0038751D]
	GetHandleVerifier [0x00784513+2899763]
	GetHandleVerifier [0x007D793D+3240797]
	GetHandleVerifier [0x005513B4+59

In [60]:
cul = pd.read_csv('China_Culture.csv')  
cul

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Hongyu, Du Mingming",Culture,20240613.0,1450.0,NaN,NaN,"Diversity, unity promote cultural progress in ...","\nPan Yue, director of the National Ethnic Aff...",4493.0,http://en.people.cn/n3/2024/0613/c98649-201810...
1,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613.0,1421.0,NaN,NaN,Why the Chinese language is gaining global app...,\nA contestant performs calligraphy during a t...,3511.0,http://en.people.cn/n3/2024/0613/c90000-201810...
2,People.cn,"Tian Yi, Zhong Wenxing",Culture,20240613.0,1359.0,NaN,NaN,Tibetan opera performance shines in Lhasa,\nArtists stage a Tibetan opera performance ne...,975.0,http://en.people.cn/n3/2024/0613/c90000-201809...
3,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613.0,1337.0,NaN,NaN,Folk melodies vitalize tourism in NW China,"YINCHUAN, June 13 (Xinhua) -- Sa Lina, 38, has...",2630.0,http://en.people.cn/n3/2024/0613/c90000-201809...
4,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613.0,1321.0,NaN,NaN,Outstanding Chinese animation showcased in France,"ANNECY, France, June 12 (Xinhua) -- Dozens of ...",1613.0,http://en.people.cn/n3/2024/0613/c90000-201808...
5,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613.0,1312.0,NaN,NaN,"Int'l forum on Xinjiang's history, future held...","URUMQI, June 12 (Xinhua) -- The International ...",1718.0,http://en.people.cn/n3/2024/0613/c90000-201808...
6,People.cn,"Hongyu, Liang Jun",Culture,20240613.0,1019.0,NaN,NaN,"Dragon boat race held in Yulin, S China's Guan...",A dragon boat invitational race was held in Yu...,622.0,http://en.people.cn/n3/2024/0613/c90000-201808...
7,People.cn,"Zhang Kaiwei, Liang Jun",Culture,20240613.0,1017.0,NaN,NaN,"""China's gift to the world"": Envoys discuss ex...","BEIJING, June 12 (Xinhua) -- ""This is a land o...",5427.0,http://en.people.cn/n3/2024/0613/c90000-201808...
8,People.cn,"Zhang Kaiwei, Liang Jun",Culture,20240613.0,1004.0,NaN,NaN,U.S. youths' visit to Chongqing Stilwell Museu...,"CHONGQING, June 12 (Xinhua) -- A group of Amer...",2821.0,http://en.people.cn/n3/2024/0613/c90000-201807...
9,People.cn,"Hongyu, Liang Jun",Culture,20240613.0,943.0,NaN,NaN,Yulin in S China's Guangxi celebrates Duanwu F...,\nPhoto shows throngs of visitors at the 14th ...,1899.0,http://en.people.cn/n3/2024/0613/c90000-201807...


In [61]:
cul['Submission Date'] = cul['Submission Date'].fillna(0).astype(int)

cul['Submission Time'] = cul['Submission Time'].fillna(0).astype(int)

cul

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Hongyu, Du Mingming",Culture,20240613,1450,NaN,NaN,"Diversity, unity promote cultural progress in ...","\nPan Yue, director of the National Ethnic Aff...",4493.0,http://en.people.cn/n3/2024/0613/c98649-201810...
1,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613,1421,NaN,NaN,Why the Chinese language is gaining global app...,\nA contestant performs calligraphy during a t...,3511.0,http://en.people.cn/n3/2024/0613/c90000-201810...
2,People.cn,"Tian Yi, Zhong Wenxing",Culture,20240613,1359,NaN,NaN,Tibetan opera performance shines in Lhasa,\nArtists stage a Tibetan opera performance ne...,975.0,http://en.people.cn/n3/2024/0613/c90000-201809...
3,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613,1337,NaN,NaN,Folk melodies vitalize tourism in NW China,"YINCHUAN, June 13 (Xinhua) -- Sa Lina, 38, has...",2630.0,http://en.people.cn/n3/2024/0613/c90000-201809...
4,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613,1321,NaN,NaN,Outstanding Chinese animation showcased in France,"ANNECY, France, June 12 (Xinhua) -- Dozens of ...",1613.0,http://en.people.cn/n3/2024/0613/c90000-201808...
5,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613,1312,NaN,NaN,"Int'l forum on Xinjiang's history, future held...","URUMQI, June 12 (Xinhua) -- The International ...",1718.0,http://en.people.cn/n3/2024/0613/c90000-201808...
6,People.cn,"Hongyu, Liang Jun",Culture,20240613,1019,NaN,NaN,"Dragon boat race held in Yulin, S China's Guan...",A dragon boat invitational race was held in Yu...,622.0,http://en.people.cn/n3/2024/0613/c90000-201808...
7,People.cn,"Zhang Kaiwei, Liang Jun",Culture,20240613,1017,NaN,NaN,"""China's gift to the world"": Envoys discuss ex...","BEIJING, June 12 (Xinhua) -- ""This is a land o...",5427.0,http://en.people.cn/n3/2024/0613/c90000-201808...
8,People.cn,"Zhang Kaiwei, Liang Jun",Culture,20240613,1004,NaN,NaN,U.S. youths' visit to Chongqing Stilwell Museu...,"CHONGQING, June 12 (Xinhua) -- A group of Amer...",2821.0,http://en.people.cn/n3/2024/0613/c90000-201807...
9,People.cn,"Hongyu, Liang Jun",Culture,20240613,943,NaN,NaN,Yulin in S China's Guangxi celebrates Duanwu F...,\nPhoto shows throngs of visitors at the 14th ...,1899.0,http://en.people.cn/n3/2024/0613/c90000-201807...


In [64]:
cul.to_csv('China_Culture.csv', index=False)



In [65]:
cul

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Hongyu, Du Mingming",Culture,20240613,1450,NaN,NaN,"Diversity, unity promote cultural progress in ...","\nPan Yue, director of the National Ethnic Aff...",4493.0,http://en.people.cn/n3/2024/0613/c98649-201810...
1,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613,1421,NaN,NaN,Why the Chinese language is gaining global app...,\nA contestant performs calligraphy during a t...,3511.0,http://en.people.cn/n3/2024/0613/c90000-201810...
2,People.cn,"Tian Yi, Zhong Wenxing",Culture,20240613,1359,NaN,NaN,Tibetan opera performance shines in Lhasa,\nArtists stage a Tibetan opera performance ne...,975.0,http://en.people.cn/n3/2024/0613/c90000-201809...
3,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613,1337,NaN,NaN,Folk melodies vitalize tourism in NW China,"YINCHUAN, June 13 (Xinhua) -- Sa Lina, 38, has...",2630.0,http://en.people.cn/n3/2024/0613/c90000-201809...
4,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613,1321,NaN,NaN,Outstanding Chinese animation showcased in France,"ANNECY, France, June 12 (Xinhua) -- Dozens of ...",1613.0,http://en.people.cn/n3/2024/0613/c90000-201808...
5,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613,1312,NaN,NaN,"Int'l forum on Xinjiang's history, future held...","URUMQI, June 12 (Xinhua) -- The International ...",1718.0,http://en.people.cn/n3/2024/0613/c90000-201808...
6,People.cn,"Hongyu, Liang Jun",Culture,20240613,1019,NaN,NaN,"Dragon boat race held in Yulin, S China's Guan...",A dragon boat invitational race was held in Yu...,622.0,http://en.people.cn/n3/2024/0613/c90000-201808...
7,People.cn,"Zhang Kaiwei, Liang Jun",Culture,20240613,1017,NaN,NaN,"""China's gift to the world"": Envoys discuss ex...","BEIJING, June 12 (Xinhua) -- ""This is a land o...",5427.0,http://en.people.cn/n3/2024/0613/c90000-201808...
8,People.cn,"Zhang Kaiwei, Liang Jun",Culture,20240613,1004,NaN,NaN,U.S. youths' visit to Chongqing Stilwell Museu...,"CHONGQING, June 12 (Xinhua) -- A group of Amer...",2821.0,http://en.people.cn/n3/2024/0613/c90000-201807...
9,People.cn,"Hongyu, Liang Jun",Culture,20240613,943,NaN,NaN,Yulin in S China's Guangxi celebrates Duanwu F...,\nPhoto shows throngs of visitors at the 14th ...,1899.0,http://en.people.cn/n3/2024/0613/c90000-201807...


In [66]:
# 39 ~ 46 행 제거
cul = cul.drop([39, 40, 41, 42, 43, 44, 45, 46])

cul

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Hongyu, Du Mingming",Culture,20240613,1450,NaN,NaN,"Diversity, unity promote cultural progress in ...","\nPan Yue, director of the National Ethnic Aff...",4493.0,http://en.people.cn/n3/2024/0613/c98649-201810...
1,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613,1421,NaN,NaN,Why the Chinese language is gaining global app...,\nA contestant performs calligraphy during a t...,3511.0,http://en.people.cn/n3/2024/0613/c90000-201810...
2,People.cn,"Tian Yi, Zhong Wenxing",Culture,20240613,1359,NaN,NaN,Tibetan opera performance shines in Lhasa,\nArtists stage a Tibetan opera performance ne...,975.0,http://en.people.cn/n3/2024/0613/c90000-201809...
3,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613,1337,NaN,NaN,Folk melodies vitalize tourism in NW China,"YINCHUAN, June 13 (Xinhua) -- Sa Lina, 38, has...",2630.0,http://en.people.cn/n3/2024/0613/c90000-201809...
4,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613,1321,NaN,NaN,Outstanding Chinese animation showcased in France,"ANNECY, France, June 12 (Xinhua) -- Dozens of ...",1613.0,http://en.people.cn/n3/2024/0613/c90000-201808...
5,People.cn,"Zhang Kaiwei, Zhong Wenxing",Culture,20240613,1312,NaN,NaN,"Int'l forum on Xinjiang's history, future held...","URUMQI, June 12 (Xinhua) -- The International ...",1718.0,http://en.people.cn/n3/2024/0613/c90000-201808...
6,People.cn,"Hongyu, Liang Jun",Culture,20240613,1019,NaN,NaN,"Dragon boat race held in Yulin, S China's Guan...",A dragon boat invitational race was held in Yu...,622.0,http://en.people.cn/n3/2024/0613/c90000-201808...
7,People.cn,"Zhang Kaiwei, Liang Jun",Culture,20240613,1017,NaN,NaN,"""China's gift to the world"": Envoys discuss ex...","BEIJING, June 12 (Xinhua) -- ""This is a land o...",5427.0,http://en.people.cn/n3/2024/0613/c90000-201808...
8,People.cn,"Zhang Kaiwei, Liang Jun",Culture,20240613,1004,NaN,NaN,U.S. youths' visit to Chongqing Stilwell Museu...,"CHONGQING, June 12 (Xinhua) -- A group of Amer...",2821.0,http://en.people.cn/n3/2024/0613/c90000-201807...
9,People.cn,"Hongyu, Liang Jun",Culture,20240613,943,NaN,NaN,Yulin in S China's Guangxi celebrates Duanwu F...,\nPhoto shows throngs of visitors at the 14th ...,1899.0,http://en.people.cn/n3/2024/0613/c90000-201807...


In [67]:
cul.to_csv('China_Culture.csv', index=False)

In [70]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import logging
import pandas as pd
import time
import re
from datetime import datetime

# 웹드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 로깅 설정
logging.basicConfig(level=logging.ERROR, filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

Press = []
Reporter = []
Category = []
SubmissionDate = [] 
SubmissionTime = []
UpdataDate = []
UpdataTime = []
titles = []
articles = []
Size = []
urls = []

url = "http://en.people.cn/"
driver.get(url)

# 정규식 패턴
time_pattern = re.compile(r'(\d{2}:\d{2}), (\w+ \d{2}, \d{4})')

for i in range(1, 11): # 페이지 무한대 설정 가능
    page = f'http://en.people.cn/90779/index{i}.html'
    driver.get(page)
    try:
        for j in range(1, 21):
            
            # 기사 페이지
            headline = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'/html/body/div/div[5]/div[1]/ul/li[{j}]/a'))
            )
            # 큰 기사 링크로 이동
            headline_href = headline.get_attribute('href')
            
            # 새 창 열기
            driver.execute_script("window.open(arguments[0]);", headline_href)
            
            # 새 창으로 전환
            driver.switch_to.window(driver.window_handles[1])
            
            # Press
            Press.append('People.cn')
            
            # reporter
            try: 
                reporter = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'editor'))
                )
                # 기자 이름 추출
                reporter_text = reporter.text
                if '(' in reporter_text and ')' in reporter_text:
                    names = reporter_text.split('(')[1].split(')')[0]
                    names = names.replace('Web editor:', '').strip()
                    reporter_names = [name.strip() for name in names.split(',')]
                    reporter_result = ', '.join(reporter_names)
                else:
                    reporter_result = ''
                Reporter.append(reporter_result)
                
            except Exception as e:
                logging.error(e)
                print(e)
                Reporter.append(None)

            # Category  
            Category.append('Sports')
            
            # Submission Date
            submission_time_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="origin cf"]/span'))
            )
            submission_time_text = submission_time_element.text
            
            match = time_pattern.search(submission_time_text)
            
            if match:
                time_str = match.group(1)
                date_str = match.group(2)
                date = datetime.strptime(date_str, '%B %d, %Y').strftime('%Y%m%d')
                
                SubmissionDate.append(int(date))
                SubmissionTime.append(int(time_str.replace(':', '')))
            else:
                SubmissionDate.append(None)
                SubmissionTime.append(None)
                
            # Last Edited Date
            UpdataDate.append(None)
            
            # Last Edited Time
            UpdataTime.append(None)
            
            # titles
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/h1'))
            )
            titles.append(title.text)
            
            # articles
            article_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="w860 d2txtCon cf"]'))
            )
            paragraphs = article_box.find_elements(By.TAG_NAME, 'p')
            article = ''
            for paragraph in paragraphs:
                article += paragraph.text + '\n'
            articles.append(article)
            
            # Size
            Size.append(len(article))
            
            # url
            urls.append(headline_href)
            
            print(f'{j}번째 기사 크롤링 완료')
            
            # 새 창 닫기
            driver.close()
            
            # 원래 창으로 돌아가기
            driver.switch_to.window(driver.window_handles[0])
            
    except Exception as e:
        logging.error(e)
        print(e)
        # 다음 단계로 넘어감 
        continue
    print(f'{i}번째 페이지 크롤링 완료')
    
    

# Press, Reporter, Category, SubmissionDate, SubmissionTime, UpdataDate, UpdataTime, titles, articles, Size, urls의 길이를 확인
array_lengths = [len(Press), len(Reporter), len(Category), len(SubmissionDate), len(SubmissionTime), len(UpdataDate), len(UpdataTime), len(titles), len(articles), len(Size), len(urls)]
max_length = max(array_lengths)

# 각 배열의 길이를 최대 길이에 맞추고, 빈 값을 NULL로 채우기
Press += [None] * (max_length - len(Press))
Reporter += [None] * (max_length - len(Reporter))
Category += [None] * (max_length - len(Category))
SubmissionDate += [None] * (max_length - len(SubmissionDate))
SubmissionTime += [None] * (max_length - len(SubmissionTime))
UpdataDate += [None] * (max_length - len(UpdataDate))
UpdataTime += [None] * (max_length - len(UpdataTime))
titles += [None] * (max_length - len(titles))
articles += [None] * (max_length - len(articles))
Size += [None] * (max_length - len(Size))
urls += [None] * (max_length - len(urls))


# 데이터 프레임 생성
politics = pd.DataFrame({
    'Press': Press,
    'Reporter': Reporter,
    'Category': Category,
    'Submission Date': SubmissionDate,
    'Submission Time': SubmissionTime,
    'Last Edited Date': UpdataDate,
    'Last Edited Time': UpdataTime,
    'Article Title': titles,
    'Article Body': articles,
    'Size': Size,
    'url': urls
})    
    

# csv 파일로 저장
politics.to_csv('China_Sports.csv', index=False)

print('크롤링 완료')


1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
1번째 페이지 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
2번째 페이지 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
3번째 페이지 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완

In [72]:
sp = pd.read_csv('China_Sports.csv')
sp

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Zhang Kaiwei, Liang Jun",Sports,20240612.0,1004.0,NaN,NaN,China sweeps Bulgaria in Women's Volleyball Na...,\nZhu Ting (top) of China competes during the ...,5143.0,http://en.people.cn/n3/2024/0612/c90000-201804...
1,People.cn,"Zhang Kaiwei, Liang Jun",Sports,20240612.0,935.0,NaN,NaN,China announces six-strong taekwondo squad for...,"BEIJING, June 11 (Xinhua) -- The Chinese Taekw...",1762.0,http://en.people.cn/n3/2024/0612/c90000-201803...
2,People.cn,"Zhang Kaiwei, Liang Jun",Sports,20240612.0,828.0,NaN,NaN,China advances in World Cup Asian qualifiers d...,\nSon Heung-min (L) of South Korea and Fang Ha...,1718.0,http://en.people.cn/n3/2024/0612/c90000-201802...
3,People.cn,"Zhang Kaiwei, Zhong Wenxing",Sports,20240611.0,1528.0,NaN,NaN,"Xinjiang sees sports venues boom, adding 30,00...","URUMQI, China, June 11 (Xinhua) -- Over the pa...",1083.0,http://en.people.cn/n3/2024/0611/c90000-201800...
4,People.cn,"Tian Yi, Zhong Wenxing",Sports,20240611.0,1314.0,NaN,NaN,HK Dragon Boat team prepares for international...,"\nThis photo released on June 10, 2024, shows ...",892.0,http://en.people.cn/n3/2024/0611/c90000-201799...
...,...,...,...,...,...,...,...,...,...,...,...
67,People.cn,"Zhang Kaiwei, Liang Jun",Sports,20240521.0,943.0,NaN,NaN,China adds 3 golds on Day 4 at Para Athletics ...,"KOBE, Japan, May 20 (Xinhua) -- Chinese athlet...",1296.0,http://en.people.cn/n3/2024/0521/c90000-201721...
68,People.cn,"Zhang Kaiwei, Liang Jun",Sports,20240521.0,922.0,NaN,NaN,China's inaugural National Fitness Competition...,"BEIJING, May 20 (Xinhua) -- China's inaugural ...",1310.0,http://en.people.cn/n3/2024/0521/c90000-201721...
69,People.cn,"Zhang Kaiwei, Liang Jun",Sports,20240521.0,916.0,NaN,NaN,China's para runner Shi breaks women's 100m T3...,"KOBE, Japan, May 20 (Xinhua) -- Chinese para a...",1332.0,http://en.people.cn/n3/2024/0521/c90000-201721...
70,People.cn,"Hongyu, Du Mingming",Sports,20240520.0,1712.0,NaN,NaN,China holds marathon to pay tribute to workers,The All-China Federation of Trade Unions has l...,1877.0,http://en.people.cn/n3/2024/0520/c98649-201719...


In [73]:
sp['Submission Date'] = sp['Submission Date'].fillna(0).astype(int)

sp['Submission Time'] = sp['Submission Time'].fillna(0).astype(int)

sp

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Zhang Kaiwei, Liang Jun",Sports,20240612,1004,NaN,NaN,China sweeps Bulgaria in Women's Volleyball Na...,\nZhu Ting (top) of China competes during the ...,5143.0,http://en.people.cn/n3/2024/0612/c90000-201804...
1,People.cn,"Zhang Kaiwei, Liang Jun",Sports,20240612,935,NaN,NaN,China announces six-strong taekwondo squad for...,"BEIJING, June 11 (Xinhua) -- The Chinese Taekw...",1762.0,http://en.people.cn/n3/2024/0612/c90000-201803...
2,People.cn,"Zhang Kaiwei, Liang Jun",Sports,20240612,828,NaN,NaN,China advances in World Cup Asian qualifiers d...,\nSon Heung-min (L) of South Korea and Fang Ha...,1718.0,http://en.people.cn/n3/2024/0612/c90000-201802...
3,People.cn,"Zhang Kaiwei, Zhong Wenxing",Sports,20240611,1528,NaN,NaN,"Xinjiang sees sports venues boom, adding 30,00...","URUMQI, China, June 11 (Xinhua) -- Over the pa...",1083.0,http://en.people.cn/n3/2024/0611/c90000-201800...
4,People.cn,"Tian Yi, Zhong Wenxing",Sports,20240611,1314,NaN,NaN,HK Dragon Boat team prepares for international...,"\nThis photo released on June 10, 2024, shows ...",892.0,http://en.people.cn/n3/2024/0611/c90000-201799...
...,...,...,...,...,...,...,...,...,...,...,...
67,People.cn,"Zhang Kaiwei, Liang Jun",Sports,20240521,943,NaN,NaN,China adds 3 golds on Day 4 at Para Athletics ...,"KOBE, Japan, May 20 (Xinhua) -- Chinese athlet...",1296.0,http://en.people.cn/n3/2024/0521/c90000-201721...
68,People.cn,"Zhang Kaiwei, Liang Jun",Sports,20240521,922,NaN,NaN,China's inaugural National Fitness Competition...,"BEIJING, May 20 (Xinhua) -- China's inaugural ...",1310.0,http://en.people.cn/n3/2024/0521/c90000-201721...
69,People.cn,"Zhang Kaiwei, Liang Jun",Sports,20240521,916,NaN,NaN,China's para runner Shi breaks women's 100m T3...,"KOBE, Japan, May 20 (Xinhua) -- Chinese para a...",1332.0,http://en.people.cn/n3/2024/0521/c90000-201721...
70,People.cn,"Hongyu, Du Mingming",Sports,20240520,1712,NaN,NaN,China holds marathon to pay tribute to workers,The All-China Federation of Trade Unions has l...,1877.0,http://en.people.cn/n3/2024/0520/c98649-201719...


In [74]:
sp.to_csv('China_Sports.csv', index=False)

In [75]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import logging
import pandas as pd
import time
import re
from datetime import datetime

# 웹드라이버 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 로깅 설정
logging.basicConfig(level=logging.ERROR, filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

Press = []
Reporter = []
Category = []
SubmissionDate = [] 
SubmissionTime = []
UpdataDate = []
UpdataTime = []
titles = []
articles = []
Size = []
urls = []

url = "http://en.people.cn/"
driver.get(url)

# 정규식 패턴
time_pattern = re.compile(r'(\d{2}:\d{2}), (\w+ \d{2}, \d{4})')

for i in range(1, 11): # 페이지 무한대 설정 가능
    page = f'http://en.people.cn/90786/index{i}.html'
    driver.get(page)
    try:
        for j in range(1, 21):
            
            # 기사 페이지
            headline = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'/html/body/div/div[5]/div[1]/ul/li[{j}]/a'))
            )
            # 큰 기사 링크로 이동
            headline_href = headline.get_attribute('href')
            
            # 새 창 열기
            driver.execute_script("window.open(arguments[0]);", headline_href)
            
            # 새 창으로 전환
            driver.switch_to.window(driver.window_handles[1])
            
            # Press
            Press.append('People.cn')
            
            # reporter
            try: 
                reporter = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'editor'))
                )
                # 기자 이름 추출
                reporter_text = reporter.text
                if '(' in reporter_text and ')' in reporter_text:
                    names = reporter_text.split('(')[1].split(')')[0]
                    names = names.replace('Web editor:', '').strip()
                    reporter_names = [name.strip() for name in names.split(',')]
                    reporter_result = ', '.join(reporter_names)
                else:
                    reporter_result = ''
                Reporter.append(reporter_result)
                
            except Exception as e:
                logging.error(e)
                print(e)
                Reporter.append(None)

            # Category  
            Category.append('Military')
            
            # Submission Date
            submission_time_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="origin cf"]/span'))
            )
            submission_time_text = submission_time_element.text
            
            match = time_pattern.search(submission_time_text)
            
            if match:
                time_str = match.group(1)
                date_str = match.group(2)
                date = datetime.strptime(date_str, '%B %d, %Y').strftime('%Y%m%d')
                
                SubmissionDate.append(int(date))
                SubmissionTime.append(int(time_str.replace(':', '')))
            else:
                SubmissionDate.append(None)
                SubmissionTime.append(None)
                
            # Last Edited Date
            UpdataDate.append(None)
            
            # Last Edited Time
            UpdataTime.append(None)
            
            # titles
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div/div[3]/h1'))
            )
            titles.append(title.text)
            
            # articles
            article_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="w860 d2txtCon cf"]'))
            )
            paragraphs = article_box.find_elements(By.TAG_NAME, 'p')
            article = ''
            for paragraph in paragraphs:
                article += paragraph.text + '\n'
            articles.append(article)
            
            # Size
            Size.append(len(article))
            
            # url
            urls.append(headline_href)
            
            print(f'{j}번째 기사 크롤링 완료')
            
            # 새 창 닫기
            driver.close()
            
            # 원래 창으로 돌아가기
            driver.switch_to.window(driver.window_handles[0])
            
    except Exception as e:
        logging.error(e)
        print(e)
        # 다음 단계로 넘어감 
        continue
    print(f'{i}번째 페이지 크롤링 완료')
    
    

# Press, Reporter, Category, SubmissionDate, SubmissionTime, UpdataDate, UpdataTime, titles, articles, Size, urls의 길이를 확인
array_lengths = [len(Press), len(Reporter), len(Category), len(SubmissionDate), len(SubmissionTime), len(UpdataDate), len(UpdataTime), len(titles), len(articles), len(Size), len(urls)]
max_length = max(array_lengths)

# 각 배열의 길이를 최대 길이에 맞추고, 빈 값을 NULL로 채우기
Press += [None] * (max_length - len(Press))
Reporter += [None] * (max_length - len(Reporter))
Category += [None] * (max_length - len(Category))
SubmissionDate += [None] * (max_length - len(SubmissionDate))
SubmissionTime += [None] * (max_length - len(SubmissionTime))
UpdataDate += [None] * (max_length - len(UpdataDate))
UpdataTime += [None] * (max_length - len(UpdataTime))
titles += [None] * (max_length - len(titles))
articles += [None] * (max_length - len(articles))
Size += [None] * (max_length - len(Size))
urls += [None] * (max_length - len(urls))


# 데이터 프레임 생성
politics = pd.DataFrame({
    'Press': Press,
    'Reporter': Reporter,
    'Category': Category,
    'Submission Date': SubmissionDate,
    'Submission Time': SubmissionTime,
    'Last Edited Date': UpdataDate,
    'Last Edited Time': UpdataTime,
    'Article Title': titles,
    'Article Body': articles,
    'Size': Size,
    'url': urls
})    
    

# csv 파일로 저장
politics.to_csv('China_Military.csv', index=False)

print('크롤링 완료')


1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
1번째 페이지 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
2번째 페이지 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완료
7번째 기사 크롤링 완료
8번째 기사 크롤링 완료
9번째 기사 크롤링 완료
10번째 기사 크롤링 완료
11번째 기사 크롤링 완료
12번째 기사 크롤링 완료
13번째 기사 크롤링 완료
14번째 기사 크롤링 완료
15번째 기사 크롤링 완료
16번째 기사 크롤링 완료
17번째 기사 크롤링 완료
18번째 기사 크롤링 완료
19번째 기사 크롤링 완료
20번째 기사 크롤링 완료
3번째 페이지 크롤링 완료
1번째 기사 크롤링 완료
2번째 기사 크롤링 완료
3번째 기사 크롤링 완료
4번째 기사 크롤링 완료
5번째 기사 크롤링 완료
6번째 기사 크롤링 완

In [76]:
mil = pd.read_csv('China_Military.csv')

mil

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Zhang Kaiwei, Zhong Wenxing",Military,20240612,1639,NaN,NaN,160 rockets fired at northern Israel -- Israel...,"JERUSALEM, June 12 (Xinhua) -- About 160 rocke...",225,http://en.people.cn/n3/2024/0612/c90000-201806...
1,People.cn,"Tian Yi, Liang Jun",Military,20240612,1029,NaN,NaN,Airmen operate ground-to-air missile at night,\nAirmen assigned to a model ground-to-air mis...,432,http://en.people.cn/n3/2024/0612/c90000-201804...
2,People.cn,"Zhang Kaiwei, Liang Jun",Military,20240612,807,NaN,NaN,Chinese defense spokesperson comments on Dutch...,"BEIJING, June 11 (Xinhua) -- A Chinese defense...",1602,http://en.people.cn/n3/2024/0612/c90000-201801...
3,People.cn,"Tian Yi, Liang Jun",Military,20240611,929,NaN,NaN,Naval coast-defense troops in multi-subject tr...,\nSoldiers assigned to a coast-defense missile...,392,http://en.people.cn/n3/2024/0611/c90000-201798...
4,People.cn,"Tian Yi, Liang Jun",Military,20240611,846,NaN,NaN,China's participation in dialogue seen as crucial,The Chinese military's participation in the Sh...,4115,http://en.people.cn/n3/2024/0611/c90000-201797...
...,...,...,...,...,...,...,...,...,...,...,...
195,People.cn,"Tian Yi, Liang Jun",Military,20240222,1002,NaN,NaN,China and India militaries to 'turn the page',\nIndian and Chinese national flags flutter si...,954,http://en.people.cn/n3/2024/0222/c90000-201359...
196,People.cn,"Zhang Kaiwei, Zhong Wenxing",Military,20240221,1633,NaN,NaN,"U.S., British warplanes strike Houthi targets ...","SANAA, Feb. 21 (Xinhua) -- U.S. and British wa...",984,http://en.people.cn/n3/2024/0221/c90000-201357...
197,People.cn,"Zhang Kaiwei, Zhong Wenxing",Military,20240221,1607,NaN,NaN,China sends new naval fleet for escort mission...,"GUANGZHOU, Feb. 21 (Xinhua) -- The 46th fleet ...",835,http://en.people.cn/n3/2024/0221/c90000-201356...
198,People.cn,"Tian Yi, Zhong Wenxing",Military,20240221,1042,NaN,NaN,Armored vehicles in maneuver training exercise,\nArmored vehicles attached to a regiment unde...,392,http://en.people.cn/n3/2024/0221/c90000-201355...


In [77]:
# 6개 csv 파일을 하나로 합치기

eco = pd.read_csv('China_Economy.csv')
soc = pd.read_csv('China_Society.csv')
wor = pd.read_csv('China_World.csv')
cul = pd.read_csv('China_Culture.csv')
spo = pd.read_csv('China_Sports.csv')
mil = pd.read_csv('China_Military.csv')

df = pd.concat([eco, soc, wor, cul, spo, mil])

df

,Press,Reporter,Category,Submission Date,Submission Time,Last Edited Date,Last Edited Time,Article Title,Article Body,Size,url
0,People.cn,"Tian Yi, Liang Jun",Economy,20240613,915,NaN,NaN,More policy support needed to boost demand ami...,\nA consumer selects tomatoes at a supermarket...,4233.0,http://en.people.cn/n3/2024/0613/c90000-201807...
1,People.cn,"Zhang Kaiwei, Liang Jun",Economy,20240613,913,NaN,NaN,World Intelligence Expo to be held in Tianjin,\nA staff member reacts with a robot at the fi...,1713.0,http://en.people.cn/n3/2024/0613/c90000-201807...
2,People.cn,"Zhang Kaiwei, Liang Jun",Economy,20240613,829,NaN,NaN,China's southern gateway region eyes high-qual...,"\nAn aerial drone photo taken on May 19, 2024 ...",2361.0,http://en.people.cn/n3/2024/0613/c90000-201806...
3,People.cn,"Zhang Kaiwei, Liang Jun",Economy,20240613,827,NaN,NaN,China now has 85 national-level tourist resorts,\nTourists visit the Zhangjiajie National Fore...,1216.0,http://en.people.cn/n3/2024/0613/c90000-201806...
4,People.cn,"Zhang Kaiwei, Liang Jun",Economy,20240613,823,NaN,NaN,EU tariffs on Chinese EVs opposed across Europe,"\nThis file photo taken on June 3, 2024 shows ...",5929.0,http://en.people.cn/n3/2024/0613/c90000-201806...
...,...,...,...,...,...,...,...,...,...,...,...
195,People.cn,"Tian Yi, Liang Jun",Military,20240222,1002,NaN,NaN,China and India militaries to 'turn the page',\nIndian and Chinese national flags flutter si...,954.0,http://en.people.cn/n3/2024/0222/c90000-201359...
196,People.cn,"Zhang Kaiwei, Zhong Wenxing",Military,20240221,1633,NaN,NaN,"U.S., British warplanes strike Houthi targets ...","SANAA, Feb. 21 (Xinhua) -- U.S. and British wa...",984.0,http://en.people.cn/n3/2024/0221/c90000-201357...
197,People.cn,"Zhang Kaiwei, Zhong Wenxing",Military,20240221,1607,NaN,NaN,China sends new naval fleet for escort mission...,"GUANGZHOU, Feb. 21 (Xinhua) -- The 46th fleet ...",835.0,http://en.people.cn/n3/2024/0221/c90000-201356...
198,People.cn,"Tian Yi, Zhong Wenxing",Military,20240221,1042,NaN,NaN,Armored vehicles in maneuver training exercise,\nArmored vehicles attached to a regiment unde...,392.0,http://en.people.cn/n3/2024/0221/c90000-201355...
